In [1]:
import os
import numpy as np
import pandas as pd
import sys
import pickle
# change this to include MNM_mcnb.py from MAC-POSTS
sys.path.append("/home/alanpi/Desktop/MAC-POSTS/side_project/network_builder")
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
from matplotlib import colors
import matplotlib
import matplotlib.dates as mdates
import datetime
import seaborn as sns
import math
from scipy import stats
from sklearn.metrics import r2_score
import random

In [2]:
import MNMAPI
from MNM_mcnb import *
from cartruckmodels import *

## Links

In [3]:
''' Refer to the slides or report for the locations of intersection A to F '''
MR_link_ID_list = [1007186,7186, # A <-> B
                   1007174,7174, # B <-> C
                   7173,1007173, # C <-> D
                   7177,1003858, # D  -> E
                   3858,1007177, # E  -> D
                   3857,1003857, # E <-> F
                   4150,3863,    # A
                   7167,         # B
                   1022916,      # C
                   1007176,      # D
                   3832,1003862,1022923 # F
                  ]

link_map = {1007186:"A -> B", 7186:"A <- B", 1007174:"B -> C", 7174:"B <- C", 7173:"C -> D", 1007173:"C <- D",
            7177:"D -> E", 1003858:"D -> E", 3858:"E -> D", 1007177:"E -> D", 3857:"E -> F", 1003857:"E <- F"}

## Scenarios

In [4]:
# Scenario 0: No traffic management or control scheme; No population and industrial/freight traffic growth from 2016 
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_0'

# # Scenario 1: West Carson street extension, ‘truck only’
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_1'

# # Scenario 2: West Carson street extension, ‘car and truck’
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_2'

# # Scenario 3: Change of development plans at the P&LE site 
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_3'

# # Scenario 4: No left truck turns on Angelina Avenue
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_4'

# # Scenario 5: Scenario 1 & 4 together
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_5'

# # Scenario 6: 20% increase of population (20% increase in car use) 
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_6'

# # Scenario 7: 10% increase in industrial/freight traffic
# data_folder = '../data/input_files_MckeesRocks_SPC_scenario_7'

## Simulation

In [5]:
data_folder_list = ['../data/input_files_MckeesRocks_SPC_scenario_0',
                    '../data/input_files_MckeesRocks_SPC_scenario_1',
                    '../data/input_files_MckeesRocks_SPC_scenario_2',
                    '../data/input_files_MckeesRocks_SPC_scenario_3',
                    '../data/input_files_MckeesRocks_SPC_scenario_4',
                    '../data/input_files_MckeesRocks_SPC_scenario_5',
                    '../data/input_files_MckeesRocks_SPC_scenario_6',
                    '../data/input_files_MckeesRocks_SPC_scenario_7',
                    
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_0',
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_1',
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_2',
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_3',
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_4',
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_5',
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_6',
                    '../data/PM_input_files_MckeesRocks_SPC_scenario_7'
                   ]

In [6]:
for data_folder in data_folder_list:
    print("==================================" + data_folder + "==================================")
    for repeat_time in range(3):
        print("------------------------------------{0}-------------------------------------------".format(repeat_time+1))
        nb = MNM_network_builder()
        nb.load_from_folder(data_folder)
        mcdta = Cartruck_Simulator(nb)
        mcdta.run_simulation(data_folder, MR_link_ID_list)


        # Travel time between nodes
        mid_intervals = np.array([i * 180 + 90 for i in range(24)])
    #     mid_intervals = np.array([10])

        car_link_tt = mcdta.get_link_car_tt(mid_intervals)
        truck_link_tt = mcdta.get_link_truck_tt(mid_intervals)

        s_car = ''
        k = 6
        for i in range(6):
            s_car += str(round(np.mean(car_link_tt[i]),2)*k) + ' '
        s_car += str(round(np.mean(car_link_tt[6]) + np.mean(car_link_tt[7]),2)*k) + ' '
        s_car += str(round(np.mean(car_link_tt[8]) + np.mean(car_link_tt[9]),2)*k) + ' '
        s_car += str(round(np.mean(car_link_tt[10]),2)*k) + ' ' + str(round(np.mean(car_link_tt[11]),2)*k)

        s_truck = ''
        for i in range(6):
            s_truck += str(round(np.mean(truck_link_tt[i]),2)*k) + ' '
        s_truck += str(round(np.mean(truck_link_tt[6]) + np.mean(truck_link_tt[7]),2)*k) + ' '
        s_truck += str(round(np.mean(truck_link_tt[8]) + np.mean(truck_link_tt[9]),2)*k) + ' '
        s_truck += str(round(np.mean(truck_link_tt[10]),2)*k) + ' ' + str(round(np.mean(truck_link_tt[11]),2)*k)

        print('Average travel times between nodes:')
        print(s_car)
        print(s_truck)


        # Waiting time at intersections
        waitting_time = mcdta.get_waiting_time_at_intersections()
        print('Average waiting times at end of links:')
        print([round(a*3, 3) for a in waitting_time])


        # Check if spill back
        spill_back = mcdta.get_link_spillback()
        print('If link spill back:')
        print(spill_back[:12])


        # Travel times to important locations
        # Downtown: 100123 -> 150120
        path_nodes1 = [3763, 3535, 1093, 178, 5862, 7169, 6798, 1726, 6040, 5703, 8236, 7423, 2827, 5711, 2864, 7585, 
                       5597, 7841, 7596, 6251, 5911, 1430, 4822, 2769, 4318, 1632, 386]
        path_links1 = [7174, 7186, 7189, 1004121, 4120, 4118, 4068, 4072, 3960, 3958, 3957, 3941, 3938, 3929, 2027, 
                       2030, 1997, 2000, 1002040, 2039, 1002043, 2047, 2056, 2163, 2148, 2139]
        # [100123 3763 3535 1093 178 5862 7169 3 5151 4025 2618 7509 341 7915 3033 3525 6024 5320 
        #  5915 6313 5062 4785 477 1119 4739 8287 4822 2769 4318 1632 6735 150120]
        tt1_car = round(np.sum(mcdta.get_path_tt_car(path_links1, mid_intervals)),2)
        tt1_truck = round(np.sum(mcdta.get_path_tt_truck(path_links1, mid_intervals)),2)

        # Airport: 100123 -> 150194
        path_nodes2 = [3763, 3535, 4083, 2370, 5755, 5788, 5215, 6003, 1213, 2187, 2335, 3820, 8321, 6596, 5907, 587, 
                       1677, 6960, 5319, 5654, 8223, 2477, 324, 6036, 5157]
        path_links2 = [7174, 1007167, 1007149, 7150, 7200, 1007203, 7202, 1007146, 7147, 1006711, 6710, 6723, 6891, 
                       7125, 1007103, 1007098, 7100, 7110, 7107, 16554, 16804, 16807, 1016546, 16545]
        # [100123 3763 3535 4083 2370 5755 5788 5215 6003 3963 6414 7543 7763 3820 8321 6596 5907 
        #  587 1677 5569 5769 7846 666 8389 4943 8134 5865 8080 150194]
        # [100123 3763 3535 4083 2370 5755 5788 5215 6003 3963 6414 7543 7763 3820 8321 6596 11 
        #  392 6818 1737 6147 4131 2584 7604 3934 7880 6704 7426 1466 4837 647 4439 3142 2454 150194]
        tt2_car = round(np.sum(mcdta.get_path_tt_car(path_links2, mid_intervals)),2)
        tt2_truck = round(np.sum(mcdta.get_path_tt_truck(path_links2, mid_intervals)),2)

        # I-79 / I-76 interchange: 100123 -> 150275
        path_nodes3 = [3763, 3535, 1093, 178, 2224, 194, 1931, 2565, 4610, 2781, 338, 4116, 634, 3950, 4804, 3050, 
                       4096, 5801, 5027]
        path_links3 = [7174, 7186, 7189, 4151, 1007356, 7357, 1007364, 7366, 1016376, 16377, 16385, 16399, 16400, 
                       7642, 7645, 7646, 7729, 1007704]
        tt3_car = round(np.sum(mcdta.get_path_tt_car(path_links3, mid_intervals)),2)
        tt3_truck = round(np.sum(mcdta.get_path_tt_truck(path_links3, mid_intervals)),2)

        print('Travel times to important locations:')
        print(tt1_car*6, tt2_car*12, tt3_car*12, tt1_truck*6, tt2_truck*12, tt3_truck*12)
        
    print("\n\n\n\n")

==================================../data/input_files_MckeesRocks_SPC_scenario_0==================================
------------------------------------1-------------------------------------------
Init simulation
Average travel times between nodes:
30.0 38.76 30.0 30.0 30.0 30.0 60.0 60.0 33.78 30.0
32.52 47.16 45.0 38.76 68.76 47.52 69.3 85.86 42.48 66.24
Average waiting times at end of links:
[15.0, 17.678, 15.003, 17.753, 17.493, 15.017, 15.047, 15.0, 15.331, 17.371, 16.167, 15.076, 15.005, 18.898, 16.893, 15.381, 15.011, 16.16, 16.369, 15.125]
If link spill back:
[0 1 0 0 0 0 0 0 0 1 0 0]
Travel times to important locations:
(1243.98, 1638.0, 1512.0, 1553.8200000000002, 2310.0, 2077.08)
------------------------------------2-------------------------------------------
Init simulation
Average travel times between nodes:
31.26 90.24 30.0 57.48 33.78 30.0 61.26 67.74 51.48 30.0
30.0 108.54 38.76 56.28 66.24 49.98 97.74 88.62 49.32 53.64
Average waiting times at end of links:
[15.59, 25.5

Init simulation
Average travel times between nodes:
44.76 60.78 31.26 51.48 63.48 30.0 69.0 75.48 41.28 39.0
47.16 64.26 36.24 59.94 82.38 56.16 92.4 95.52 34.98 59.22
Average waiting times at end of links:
[17.956, 21.761, 16.997, 25.201, 24.141, 17.758, 18.845, 26.476, 20.745, 25.657, 18.356, 28.973, 16.782, 20.902, 22.179, 16.066, 15.67, 19.405, 19.813, 15.855]
If link spill back:
[1 1 0 1 1 1 1 1 1 1 1 1]
Travel times to important locations:
(1372.08, 1764.0, 1633.8000000000002, 1721.6999999999998, 2506.08, 2308.68)
------------------------------------3-------------------------------------------
Init simulation
Average travel times between nodes:
30.0 80.52 30.0 52.74 42.48 30.0 67.5 61.26 42.78 30.0
31.26 87.66 36.24 55.56 63.78 46.26 99.84 78.06 59.34 56.82
Average waiting times at end of links:
[15.448, 22.118, 15.929, 23.3, 19.077, 15.533, 19.895, 15.0, 15.265, 15.699, 26.2, 15.0, 16.398, 23.79, 23.475, 15.907, 16.463, 23.521, 23.282, 19.603]
If link spill back:
[0 1 1 1 1 0 1 

Init simulation
Average travel times between nodes:
52.26 75.78 36.24 48.78 73.5 30.0 97.26 60.0 45.48 60.18
55.44 76.74 63.6 50.58 149.64 71.28 128.82 108.0 73.08 91.62
Average waiting times at end of links:
[20.412, 22.471, 22.677, 23.838, 32.168, 17.218, 28.244, 33.809, 15.007, 15.866, 25.151, 37.318, 21.035, 23.694, 23.163, 16.663, 16.871, 17.946, 25.418, 22.347]
If link spill back:
[1 1 1 1 1 1 1 1 0 0 1 1]
Travel times to important locations:
(1446.18, 1947.84, 5963.88, 1935.1799999999998, 3065.04, 6726.719999999999)





==================================../data/PM_input_files_MckeesRocks_SPC_scenario_1==================================
------------------------------------1-------------------------------------------
Init simulation
Average travel times between nodes:
47.28 54.78 34.98 61.74 147.24 42.48 76.74 85.74 49.26 32.52
53.4 69.24 48.0 74.88 203.22 68.7 132.3 139.68 86.7 72.96
Average waiting times at end of links:
[20.635, 21.562, 22.648, 29.996, 41.613, 23.642, 24.259, 

Init simulation
Average travel times between nodes:
32.52 88.98 30.0 51.48 63.48 50.52 84.48 67.74 65.76 49.26
40.02 98.52 65.52 56.82 148.62 117.42 122.34 115.92 94.86 109.62
Average waiting times at end of links:
[17.725, 25.067, 18.531, 26.065, 30.618, 23.185, 28.836, 28.789, 19.309, 23.897, 38.936, 32.427, 19.461, 25.818, 23.692, 18.418, 19.482, 35.117, 24.413, 21.508]
If link spill back:
[0 1 0 1 1 1 1 1 1 1 1 1]
Travel times to important locations:
(1737.2400000000002, 1814.3999999999999, 6247.68, 2303.2799999999997, 3694.08, 7273.799999999999)
------------------------------------2-------------------------------------------
Init simulation
Average travel times between nodes:
38.76 88.98 34.98 46.26 99.48 30.0 72.78 83.28 37.5 37.74
45.0 92.64 65.58 47.52 149.52 53.76 115.5 123.6 58.08 78.54
Average waiting times at end of links:
[19.348, 26.176, 19.797, 22.919, 32.671, 15.84, 22.453, 21.349, 20.971, 31.062, 16.893, 28.227, 20.606, 25.828, 23.903, 16.18, 17.68, 21.797, 21.099, 21.

## Find path link ids from node ids

In [7]:
graph_file_name = os.path.join('..', 'data', 'input_files_MckeesRocks_SPC_scenario_0', 'Snap_graph')

node_link_map = {}
with open(graph_file_name, 'r') as infile:
    for line in infile.readlines():
        if line[0] != '#':
            data = line.rstrip('\n').split(' ')
            node_link_map[data[1]+' '+data[2]] = data[0]

# Downtown: 100123 -> 150120
path_nodes1 = [3763, 3535, 1093, 178, 5862, 7169, 6798, 1726, 6040, 5703, 8236, 7423, 2827, 5711, 2864, 7585, 
 5597, 7841, 7596, 6251, 5911, 1430, 4822, 2769, 4318, 1632, 386]
s = ''
for i in range(len(path_nodes1) - 1):
    s += node_link_map[str(path_nodes1[i])+' '+str(path_nodes1[i+1])] + ', '
s = s[:-2]
print(s)


# Airport: 100123 -> 150194
path_nodes2 = [3763, 3535, 4083, 2370, 5755, 5788, 5215, 6003, 1213, 2187, 2335, 3820, 8321, 6596, 5907, 587, 
 1677, 6960, 5319, 5654, 8223, 2477, 324, 6036, 5157]
s = ''
for i in range(len(path_nodes2) - 1):
    s += node_link_map[str(path_nodes2[i])+' '+str(path_nodes2[i+1])] + ', '
s = s[:-2]
print(s)

# I-79 / I-76 interchange: 100123 -> 150275
path_nodes3 = [3763, 3535, 1093, 178, 2224, 194, 1931, 2565, 4610, 2781, 338, 4116, 634, 3950, 4804, 3050, 
 4096, 5801, 5027]
s = ''
for i in range(len(path_nodes3) - 1):
    s += node_link_map[str(path_nodes3[i])+' '+str(path_nodes3[i+1])] + ', '
s = s[:-2]
print(s)

7174, 7186, 7189, 1004121, 4120, 4118, 4068, 4072, 3960, 3958, 3957, 3941, 3938, 3929, 2027, 2030, 1997, 2000, 1002040, 2039, 1002043, 2047, 2056, 2163, 2148, 2139
7174, 1007167, 1007149, 7150, 7200, 1007203, 7202, 1007146, 7147, 1006711, 6710, 6723, 6891, 7125, 1007103, 1007098, 7100, 7110, 7107, 16554, 16804, 16807, 1016546, 16545
7174, 7186, 7189, 4151, 1007356, 7357, 1007364, 7366, 1016376, 16377, 16385, 16399, 16400, 7642, 7645, 7646, 7729, 1007704
